In [6]:
import cv2
import time
import numpy as np
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import HandTrackingModule as htm

In [7]:
wCam, hCam = 640, 480

In [8]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

In [9]:
detector = htm.handDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

In [10]:
while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        
        
        
        cv2.circle(img, (x1, y1), 10, (255, 0, 0), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 10, (255, 255, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        length = math.hypot(x2 - x1, y2 - y1)
        # print(length)
        
        #hand range  195 - 25
        # volume range  -63.5 - 0
        
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volPer = np.interp(length, [50, 300], [0, 100])
        volBar = np.interp(length, [50, 300], [400, 150])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)
        
        if length < 25:
            cv2.circle(img, (cx, cy), 10, (0, 0, 0), cv2.FILLED)
    
    
    cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)
    cTime = time.time()
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, f': {int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    cv2.imshow('img', img)
    cv2.waitKey(1)

29 -63.5
38 -63.5
36 -63.5
51 -63.08759594887345
48 -63.5
84 -54.767456520515346
135 -41.81605461847055
155 -36.69177169246892
160 -35.53143503884112
179 -30.708465270997948
181 -30.214774851045902
196 -26.322781152113144
196 -26.399803655005535
206 -23.85381053027833
225 -19.049435558342914
234 -16.744464748856224
233 -17.013094623895057
251 -12.405029101033364
248 -13.166533809412002
252 -12.106885954885918
251 -12.373181028661627
255 -11.318037945820414
255 -11.302627202636316
257 -10.795616507759974
255 -11.318037945820414
251 -12.299940219120295
249 -12.854111009474337
247 -13.361303482646932
245 -13.895399816706949
244 -14.10699978258419
243 -14.37564864877271
245 -13.895399816706949
242 -14.614034683216993
241 -14.922269689551975
245 -13.882457493896638
251 -12.346905697530993
246 -13.641702708593485
244 -14.136099220239146
253 -11.877289547159165
252 -12.131048861402448
260 -10.142417725139218
266 -8.612609312091358
263 -9.390274211010265
273 -6.811495447732852
273 -6.811495447

KeyboardInterrupt: 